In [ ]:
# import libraries
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from shapely import wkt
import itertools
from street_features import *

In [ ]:
## Import Seattle street and map data
Seattle_poly = ox.geocode_to_gdf('Seattle, Washington')

In [ ]:
Seattle_poly.head(1)

In [ ]:
# grab street data (roads and intersections) for entire city
sea_streets = ox.graph_from_place('Seattle, Washington', network_type = 'drive')

In [ ]:
nodes, edges = ox.graph_to_gdfs(sea_streets)
nodes.head()

In [ ]:
#remove doubled road IDs, this is only 20 roads out of 22335, but they cause subsetting problems
#sea_rds = remove_doubleID_streets(edges)
sea_rds = edges

In [ ]:
# grab subsets of roadtypes
sea_highways = sea_rds[sea_rds.highway == 'motorway']
sea_primary = sea_rds[sea_rds.highway == 'primary']
sea_secondary = sea_rds[sea_rds.highway == 'secondary']
sea_resid = sea_rds[sea_rds.highway == 'residential']

In [ ]:
## Import walk_score data, create shapely points, and convert to geopandas dataframe
walk_df = pd.read_csv('Data/master_combined.csv', index_col=0)

geometry = [Point(xy) for xy in zip(walk_df.lon, walk_df.lat)]
walk_gdf = gpd.GeoDataFrame(walk_df, geometry=geometry, crs='epsg:4326')
walk_gdf.head(1)

In [ ]:
#subset walk_score data in Seattle polygon grid
EDF_points = gpd.sjoin(walk_gdf, Seattle_poly, how="inner")
EDF_points = EDF_points.rename(index=str)

EDF_points = EDF_points.reset_index(drop=True)
EDF_points = EDF_points.drop(['index_right', 'bbox_east', 'bbox_north', 'bbox_south', 'bbox_west'], axis=1)
EDF_points.head(3)

In [112]:
EDF_points.shape

(881, 31)

Census Data

In [142]:
WA_tracts = pd.read_csv('Data/all_demographics.csv') 

In [143]:
WA_tracts['geometry'] = WA_tracts['geometry'].apply(wkt.loads)
WA_tracts = gpd.GeoDataFrame(WA_tracts, geometry = WA_tracts['geometry'], crs='epsg:2927')
WA_tracts = WA_tracts.drop(['Unnamed: 0'], axis = 1)

In [144]:
WA_tracts.head()

,GEOID,population,AREA_ACRES,tract,geometry
0,53033000101,3759,102.503713,101,"POLYGON ((1198280.286 879815.173, 1198288.869 ..."
1,53033000102,4321,596.442465,102,"POLYGON ((1198268.837 877820.803, 1198710.196 ..."
2,53033000201,4416,446.424496,201,"POLYGON ((1191537.977 876074.777, 1191548.225 ..."
3,53033000202,4099,365.710684,202,"POLYGON ((1194973.723 880600.170, 1195306.262 ..."
4,53033000300,2820,299.225793,300,"POLYGON ((1186323.563 880876.512, 1186351.366 ..."


In [145]:
WA_tracts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   GEOID       180 non-null    int64   
 1   population  180 non-null    int64   
 2   AREA_ACRES  180 non-null    float64 
 3   tract       180 non-null    int64   
 4   geometry    180 non-null    geometry
dtypes: float64(1), geometry(1), int64(3)
memory usage: 7.2 KB


In [147]:
# calculate area and population density
WA_tracts['Land_Area_Km2'] = WA_tracts['AREA_ACRES'] * 0.00404686
WA_tracts['pop_den'] = WA_tracts['population'] / WA_tracts['Land_Area_Km2']

WA_tracts['pop_den'] = WA_tracts['pop_den'].fillna(0)

WA_tracts.head(3)

,GEOID,population,AREA_ACRES,tract,geometry,Land_Area_Km2,pop_den
0,53033000101,3759,102.503713,101,"POLYGON ((1198280.286 879815.173, 1198288.869 ...",0.414818,9061.801571
1,53033000102,4321,596.442465,102,"POLYGON ((1198268.837 877820.803, 1198710.196 ...",2.413719,1790.183415
2,53033000201,4416,446.424496,201,"POLYGON ((1191537.977 876074.777, 1191548.225 ...",1.806617,2444.347050


## Seattle Zoning Data

In [ ]:
## import zoning data
sea_zones = gpd.read_file('Zoning Data/Zoning_Detailed.shp')

In [ ]:
print('Seattle Zoning Default CRS:', sea_zones.crs)

In [ ]:
sea_zones.info()

In [ ]:
# drop extraneous columns
extra_cols = ['OVERLAY', 'CONTRACT', 'OBJECTID', 'ORDINANCE', 'EFFECTIVE','SHORELINE',
              'MHA', 'MHA_VALUE', 'OVERLAY_PR', 'LIGHTRAIL_',
             'EFFECTIVE_', 'PEDESTRI_1', 'SHORELINE_', 'MIO_NAME', 'IZ',
             'ZONING_PRE', 'CONTRACT_P', 'HISTORIC_P', 'SHAPE_Leng', 'ORDINANCE_', 'ZONING_DES','DETAIL_DES', 'CATEGORY_D', 'ZONELUT', 'ZONING', 'BASE_ZONE', 'SHAPE_Area']

sea_zones = sea_zones.drop(extra_cols, axis = 1)
sea_zones.info()

In [149]:
## remove null polygons (causes issues with merges)

sea_zones = sea_zones[sea_zones.geometry.notnull()]

sea_zones.head(3)

,ZONEID,HISTORIC,PEDESTRIAN,LIGHTRAIL,CLASS_DESC,geometry,zone
0,3837,None,None,None,Downtown,"POLYGON ((551613.137 5271886.718, 551662.403 5...",commercial
1,4194,None,None,None,Single Family,"POLYGON ((549388.001 5284938.916, 549387.356 5...",residential
2,4422,None,None,None,Multi-Family,"POLYGON ((553277.906 5281916.508, 553278.157 5...",residential


In [ ]:
sea_zones.shape

In [ ]:
sea_zones['CLASS_DESC'].value_counts()

In [ ]:
# lots of zoning information that needs to be cleaned into distinct groups
# Groups: commercial, industrial, residential

sea_zones['zone'] = sea_zones['CLASS_DESC']


## zoning designations to be generalized across the Bay Area
residential = ['Multi-Family', 'Single Family', 'Master Planned Community']
commercial = ['Commercial/Mixed Use', 'Downtown']
industrial = ['Major Institutions', 'Manufacturing/Industrial']


## simplify down to the above zoning designations
sea_zones['zone'] = sea_zones['zone'].replace(dict.fromkeys(residential, 'residential'))
sea_zones['zone'] = sea_zones['zone'].replace(dict.fromkeys(commercial, 'commercial'))
sea_zones['zone'] = sea_zones['zone'].replace(dict.fromkeys(industrial, 'industrial'))

print(sea_zones['zone'].unique())

# Reprojecting

In [104]:
EDF_points = EDF_points.to_crs('epsg:32610')
EDF_points.crs

<Derived Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [105]:
sea_rds = sea_rds.to_crs('epsg:32610')

In [107]:
sea_zones = ea_zones.to_crs('epsg:32610')

# Distances to Roadways

In [95]:
#calculate closest road_type for each point within 300 meter buffer
#EDF_points['road_type'] = EDF_points['geometry'].apply(find_closest_road,roads = sea_rds, buffer_dist = 0.0030)

In [ ]:
# Cleaning road categories ...
#EDF_points['road_type'] = EDF_points['road_type'].str.replace('_link', '')
#EDF_points['road_type'] = np.where(EDF_points['road_type'] == 'trunk', 'secondary', EDF_points['road_type'])
#EDF_points['road_type'] = np.where(EDF_points['road_type'] == 'living_street', 'residential', EDF_points['road_type'])
#EDF_points['road_type'] = np.where(EDF_points['road_type'] == 'a', 'unclassified', EDF_points['road_type'])
#EDF_points['road_type'] = np.where(EDF_points['road_type'] == 'razed', 'unclassified', EDF_points['road_type'])

In [98]:
#EDF_points['road_type'].value_counts()

outside_area    881
Name: road_type, dtype: int64

In [99]:
## Calculate distance to nearest major roadway

# much faster to do the re-projection to meters outside of the apply function
EDF_utm = EDF_points.to_crs('epsg:32610').copy()
highway_utm = sea_highways.to_crs('epsg:32610').copy()
primary_utm = sea_primary.to_crs('epsg:32610').copy()
secondary_utm = sea_secondary.to_crs('epsg:32610').copy()

EDF_points['closest_highway'] = EDF_utm['geometry'].apply(distance_to_roadway, roadway = highway_utm)
EDF_points['closest_primary'] = EDF_utm['geometry'].apply(distance_to_roadway, roadway = primary_utm)
EDF_points['closest_secondary'] = EDF_utm['geometry'].apply(distance_to_roadway, roadway = secondary_utm)

In [100]:
## Calculate distance to nearest intersection and traffic signals
nodes_utm = nodes.to_crs('epsg:32610').copy() #re-project as above
signals = nodes_utm[nodes_utm['highway'] == 'traffic_signals']

EDF_points['corner_dist'] = EDF_utm['geometry'].apply(nearest_intersection,
                                                            intersections = nodes_utm['geometry'])

EDF_points['signal_dist'] = EDF_utm['geometry'].apply(nearest_intersection, 
                                                               intersections = signals['geometry'])

In [101]:
EDF_points.head()

,lat_left,lon_left,GEOID,restaurant_count,school_count,park_count,bus_station_count,supermarket_count,pub_count,parkwide_count,...,display_name,class,type,importance,road_type,closest_highway,closest_primary,closest_secondary,corner_dist,signal_dist
0,47.648701,-122.362286,53033005901,10,1,5,3,1,9,10,...,"Seattle, King County, Washington, United States",boundary,administrative,0.892979,outside_area,2939.728537,259.671825,565.130723,57.411160,261.343250
1,47.694938,-122.304282,53033002100,2,5,2,7,0,2,10,...,"Seattle, King County, Washington, United States",boundary,administrative,0.892979,outside_area,1355.193429,101.711773,35.082356,8.096012,307.197490
2,47.558749,-122.308561,53033010402,2,4,3,3,1,0,10,...,"Seattle, King County, Washington, United States",boundary,administrative,0.892979,outside_area,863.994454,668.918353,208.480125,56.139744,646.828320
3,47.617798,-122.353811,53033008003,10,2,10,3,5,10,10,...,"Seattle, King County, Washington, United States",boundary,administrative,0.892979,outside_area,660.618804,85.004984,10.673905,12.270808,113.035911
4,47.654186,-122.356103,53033004800,10,2,9,3,2,10,10,...,"Seattle, King County, Washington, United States",boundary,administrative,0.892979,outside_area,2514.772006,475.335005,3.226257,37.898277,247.892422


## Merging

In [150]:
sea_zones.shape

(3554, 7)

In [151]:
EDF_points.shape

(881, 31)

In [152]:
# left merge with the zoning data
walk_all = gpd.sjoin(EDF_points, sea_zones, how='left')
walk_all.shape

(881, 38)

In [153]:
walk_all.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 881 entries, 0 to 880
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   lat_left              881 non-null    float64 
 1   lon_left              881 non-null    float64 
 2   GEOID                 881 non-null    int64   
 3   restaurant_count      881 non-null    int64   
 4   school_count          881 non-null    int64   
 5   park_count            881 non-null    int64   
 6   bus_station_count     881 non-null    int64   
 7   supermarket_count     881 non-null    int64   
 8   pub_count             881 non-null    int64   
 9   parkwide_count        881 non-null    int64   
 10  restaurantwide_count  881 non-null    int64   
 11  intersection_count    881 non-null    int64   
 12  streets_per_node_avg  881 non-null    float64 
 13  circuity_avg          881 non-null    float64 
 14  street_length_avg     881 non-null    float64 
 15

In [154]:
walk_all = walk_all.dropna(axis=0,subset=['CLASS_DESC'])

In [155]:
# remove unneeded columsn
walk_all = walk_all.drop(['index_right','ZONEID'], axis = 1)


walk_all.head(3)

,lat_left,lon_left,GEOID,restaurant_count,school_count,park_count,bus_station_count,supermarket_count,pub_count,parkwide_count,...,closest_highway,closest_primary,closest_secondary,corner_dist,signal_dist,HISTORIC,PEDESTRIAN,LIGHTRAIL,CLASS_DESC,zone
0,47.648701,-122.362286,53033005901,10,1,5,3,1,9,10,...,2939.728537,259.671825,565.130723,57.411160,261.34325,None,None,None,Major Institutions,industrial
1,47.694938,-122.304282,53033002100,2,5,2,7,0,2,10,...,1355.193429,101.711773,35.082356,8.096012,307.19749,None,None,None,Multi-Family,residential
2,47.558749,-122.308561,53033010402,2,4,3,3,1,0,10,...,863.994454,668.918353,208.480125,56.139744,646.82832,None,None,None,Single Family,residential


In [156]:
WA_tracts.shape

(180, 7)

In [157]:
WA_tracts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   GEOID          180 non-null    int64   
 1   population     180 non-null    int64   
 2   AREA_ACRES     180 non-null    float64 
 3   tract          180 non-null    int64   
 4   geometry       180 non-null    geometry
 5   Land_Area_Km2  180 non-null    float64 
 6   pop_den        180 non-null    float64 
dtypes: float64(3), geometry(1), int64(3)
memory usage: 10.0 KB


In [158]:
WA_tracts.head()

,GEOID,population,AREA_ACRES,tract,geometry,Land_Area_Km2,pop_den
0,53033000101,3759,102.503713,101,"POLYGON ((1198280.286 879815.173, 1198288.869 ...",0.414818,9061.801571
1,53033000102,4321,596.442465,102,"POLYGON ((1198268.837 877820.803, 1198710.196 ...",2.413719,1790.183415
2,53033000201,4416,446.424496,201,"POLYGON ((1191537.977 876074.777, 1191548.225 ...",1.806617,2444.347050
3,53033000202,4099,365.710684,202,"POLYGON ((1194973.723 880600.170, 1195306.262 ...",1.479980,2769.632138
4,53033000300,2820,299.225793,300,"POLYGON ((1186323.563 880876.512, 1186351.366 ...",1.210925,2328.798436


In [159]:
walk_all.head()

,lat_left,lon_left,GEOID,restaurant_count,school_count,park_count,bus_station_count,supermarket_count,pub_count,parkwide_count,...,closest_highway,closest_primary,closest_secondary,corner_dist,signal_dist,HISTORIC,PEDESTRIAN,LIGHTRAIL,CLASS_DESC,zone
0,47.648701,-122.362286,53033005901,10,1,5,3,1,9,10,...,2939.728537,259.671825,565.130723,57.411160,261.343250,None,None,None,Major Institutions,industrial
1,47.694938,-122.304282,53033002100,2,5,2,7,0,2,10,...,1355.193429,101.711773,35.082356,8.096012,307.197490,None,None,None,Multi-Family,residential
2,47.558749,-122.308561,53033010402,2,4,3,3,1,0,10,...,863.994454,668.918353,208.480125,56.139744,646.828320,None,None,None,Single Family,residential
3,47.617798,-122.353811,53033008003,10,2,10,3,5,10,10,...,660.618804,85.004984,10.673905,12.270808,113.035911,None,None,None,Downtown,commercial
4,47.654186,-122.356103,53033004800,10,2,9,3,2,10,10,...,2514.772006,475.335005,3.226257,37.898277,247.892422,None,None,None,Multi-Family,residential


In [160]:
#merge with the census data on GEOID
walk_all = pd.merge(walk_all, WA_tracts, on='GEOID')
walk_all.head(1)

,lat_left,lon_left,GEOID,restaurant_count,school_count,park_count,bus_station_count,supermarket_count,pub_count,parkwide_count,...,PEDESTRIAN,LIGHTRAIL,CLASS_DESC,zone,population,AREA_ACRES,tract,geometry_y,Land_Area_Km2,pop_den
0,47.648701,-122.362286,53033005901,10,1,5,3,1,9,10,...,None,None,Major Institutions,industrial,3570,235.297362,5901,"POLYGON ((1178008.964 852317.994, 1178009.710 ...",0.952215,3749.151383


In [168]:

walk_all = walk_all.rename(index=str, columns = {'geometry_x': 'geometry'})
walk_all[['geometry','geometry_y']].head()

,geometry,geometry_y
0,POINT (547892.556 5277452.929),"POLYGON ((1178008.964 852317.994, 1178009.710 ..."
1,POINT (547045.419 5278497.010),"POLYGON ((1178008.964 852317.994, 1178009.710 ..."
2,POINT (547280.948 5277965.975),"POLYGON ((1178008.964 852317.994, 1178009.710 ..."
3,POINT (552202.505 5282629.192),"POLYGON ((1195070.948 864684.767, 1195403.963 ..."
4,POINT (552144.769 5282123.857),"POLYGON ((1195070.948 864684.767, 1195403.963 ..."


In [169]:
## Calculate distance to industrial zone
#re-projections
walk_utm = walk_all.to_crs('epsg:32610').copy()
zones_utm = sea_zones.to_crs('epsg:32610').copy()
industry = zones_utm[zones_utm.zone == 'industrial']
commercial = zones_utm[zones_utm.zone == 'commercial']
residential = zones_utm[zones_utm.zone == 'residential']

In [170]:
#calculations
walk_all['industry_dist'] = walk_utm['geometry'].apply(distance_to_zoning, zone = industry)
walk_all['commercial_dist'] = walk_utm['geometry'].apply(distance_to_zoning, zone = commercial)
walk_all['residential_dist'] = walk_utm['geometry'].apply(distance_to_zoning, zone = residential)

In [171]:
walk_all.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 629 entries, 0 to 628
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   lat_left              629 non-null    float64 
 1   lon_left              629 non-null    float64 
 2   GEOID                 629 non-null    int64   
 3   restaurant_count      629 non-null    int64   
 4   school_count          629 non-null    int64   
 5   park_count            629 non-null    int64   
 6   bus_station_count     629 non-null    int64   
 7   supermarket_count     629 non-null    int64   
 8   pub_count             629 non-null    int64   
 9   parkwide_count        629 non-null    int64   
 10  restaurantwide_count  629 non-null    int64   
 11  intersection_count    629 non-null    int64   
 12  streets_per_node_avg  629 non-null    float64 
 13  circuity_avg          629 non-null    float64 
 14  street_length_avg     629 non-null    float64 
 15  geo

In [172]:
walk_all.to_csv('Data/master_features.csv')